In [ ]:
import cv2  
import math 
import csv
import numpy as np
from collections import defaultdict, Counter
from ultralytics import YOLO
import os
import random
import shutil
import torch



In [ ]:

# Carpeta base del dataset
base_dir = r"C:\Users\javie\Pictures\plates\plates"


# Carpeta donde están los labels  los txt

source_labels = os.path.join(base_dir, "txt")

images = [f for f in os.listdir(base_dir) if f.endswith((".jpg", ".png"))]

# Mezclar aleatoriamente
random.shuffle(images)

# Porcentajes de división
train_split = 0.7
val_split = 0.2

num_total = len(images)
num_train = int(num_total * train_split)
num_val = int(num_total * val_split)
num_test = num_total - num_train - num_val

# Carpeta destino
dest = "\Parte_A"
dest_base = os.path.join(dest, "TGC_RBNW")

# Crear carpetas si no existen
for split in ["train", "val", "test"]:
    os.makedirs(os.path.join(dest_base, split, "images"), exist_ok=True)
    os.makedirs(os.path.join(dest_base, split, "labels"), exist_ok=True)

# Función para mover imagen y su label
def copy_files(split_name, files):
    for img_file in files:
        label_file = img_file.rsplit(".", 1)[0] + ".txt"
        src_img_path = os.path.join(base_dir, img_file)
        src_label_path = os.path.join(source_labels, label_file)

        # Solo copiar si existen ambos (imagen + txt)
        if not os.path.exists(src_img_path):
            print(f" No se encontró imagen: {img_file}")
            continue
        if not os.path.exists(src_label_path):
            print(f" Omitida {img_file}: no tiene label.")
            continue

        # Destinos
        dst_img_path = os.path.join(dest_base, split_name, "images", img_file)
        dst_label_path = os.path.join(dest_base, split_name, "labels", label_file)

        # Copiar archivos
        shutil.copy(src_img_path, dst_img_path)
        shutil.copy(src_label_path, dst_label_path)
        print(f" Copiado {img_file} y su label a {split_name}/")

copy_files("train", images[:num_train])
copy_files("val", images[num_train:num_train + num_val])
copy_files("test", images[num_train + num_val:])

print("\n División completada correctamente (solo imágenes con labels).")

In [ ]:

print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA version (compiled):", torch.version.cuda)
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")

model = YOLO('yolo11n.pt')  

model.train(
    data=r"data.yaml",  # ruta a tu YAML
    epochs=50,       # número de épocas (puedes ajustar)
    imgsz=720,        # tamaño de imagen
    batch=16,         # tamaño de lote
    name="matricula_yolo11n",  # nombre de la carpeta de resultados
    workers=1,        # hilos de carga (baja si entrenas en Windows)
    device=0          # GPU (usa "cpu" si no tienes CUDA)
)


In [ ]:

model_general = YOLO('yolo11n.pt')
model_plate = YOLO(
    r"runs\detect\matricula_yolo11n\weights\best.pt"
)

classNames = ["person", "bicycle", "car"]

video_path = r"C0142.MP4"
vid = cv2.VideoCapture(video_path)

output_video_path = (
    r"resultados_video\resultados_deteccion.mp4"
)
output_csv_path = (
    r"resultados_video\detecciones.csv"
)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = int(vid.get(cv2.CAP_PROP_FPS))
width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

frame_count = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
print(f" Procesando video ({frame_count} fotogramas)...\n")

with open(output_csv_path, mode='w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow([
        "frame", "tipo_objeto", "confianza", "id_tracking",
        "x1", "y1", "x2", "y2",
        "matricula_detectada", "conf_matricula",
        "mx1", "my1", "mx2", "my2", "texto_matricula"
    ])

    conteo = Counter()
    frame_num = 0

    while True:
        ret, frame = vid.read()
        if not ret:
            break
        frame_num += 1

        results_general = model_general.track(frame, persist=True, classes=[0, 2])
        if results_general and len(results_general[0].boxes) > 0:
            for box in results_general[0].boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                conf = float(box.conf[0])
                cls = int(box.cls[0])
                label = classNames[cls]
                track_id = int(box.id[0]) if box.id is not None else -1
                
                car_crop = None
                if label == "car":
                    car_crop = frame[y1:y2, x1:x2]
                color = (0, 255, 0) if label == "person" else (255, 0, 0)
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, f"{label} {conf:.2f} ID:{track_id}", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

                conteo[label] += 1
                matricula_detectada = ""
                conf_matricula = ""
                mx1 = my1 = mx2 = my2 = ""
                texto_matricula = ""

                
                if car_crop is not None and car_crop.size > 0:
                    results_plate = model_plate.predict(source=car_crop, device=0, verbose=False)
                    for p in results_plate:
                        for b in p.boxes:
                            px1, py1, px2, py2 = map(int, b.xyxy[0])
                            pconf = float(b.conf[0])
                            mx1, my1, mx2, my2 = x1 + px1, y1 + py1, x1 + px2, y1 + py2
                            matricula_detectada = "matricula"
                            conf_matricula = round(pconf, 2)
                            cv2.rectangle(frame, (mx1, my1), (mx2, my2), (0, 255, 255), 2)
                            cv2.putText(frame, f"plate {pconf:.2f}", (mx1, my1 - 5),
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
                            break


                writer.writerow([
                    frame_num, label, round(conf, 2), track_id,
                    x1, y1, x2, y2,
                    matricula_detectada, conf_matricula,
                    mx1, my1, mx2, my2, texto_matricula
                ])

        out.write(frame)
        if frame_num % 10 == 0:
            print(f"Procesados {frame_num}/{frame_count} fotogramas...")

vid.release()
out.release()
cv2.destroyAllWindows()

print("\n PROCESAMIENTO COMPLETADO")
print(f" Video guardado en: {output_video_path}")
print(f" CSV guardado en: {output_csv_path}")
print(f" Conteo total: {dict(conteo)}")


In [ ]:
import cv2
import csv
from collections import defaultdict, Counter
from ultralytics import YOLO

# --- MODELOS ---
model_general = YOLO('yolo11n.pt')
model_plate = YOLO(r"runs\detect\matricula_yolo11n\weights\best.pt")
classNames = ["person", "bicycle", "car"]

# --- RUTAS ---
video_path = r"video_matriculas.mp4"
output_video_path = r"resultados_video\resultados_deteccion_segundo_video.mp4"
output_csv_path = r"resultados_video\detecciones_segundo_video.csv"

# --- PARÁMETROS DEL VÍDEO ---
vid = cv2.VideoCapture(video_path)
fps = int(vid.get(cv2.CAP_PROP_FPS))
width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_count = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
print(f" Procesando video ({frame_count} fotogramas)...\n")

# --- FASE 1: DETECCIÓN Y GUARDADO DE DATOS ---
with open(output_csv_path, mode='w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow([
        "frame", "tipo_objeto", "confianza", "id_tracking",
        "x1", "y1", "x2", "y2",
        "matricula_detectada", "conf_matricula",
        "mx1", "my1", "mx2", "my2", "texto_matricula"
    ])

    conteo = Counter()
    frame_num = 0

    while True:
        ret, frame = vid.read()
        if not ret:
            break
        frame_num += 1

        results_general = model_general.track(frame, persist=True, classes=[0, 2])

        if results_general and len(results_general[0].boxes) > 0:
            for box in results_general[0].boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                conf = float(box.conf[0])
                cls = int(box.cls[0])
                label = classNames[cls]
                track_id = int(box.id[0]) if box.id is not None else -1

                conteo[label] += 1
                matricula_detectada = ""
                conf_matricula = ""
                mx1 = my1 = mx2 = my2 = ""
                texto_matricula = ""

                if label == "car":
                    car_crop = frame[y1:y2, x1:x2].copy()
                    if car_crop.size > 0:
                        results_plate = model_plate.predict(source=car_crop, device=0, verbose=False)
                        for p in results_plate:
                            for b in p.boxes:
                                px1, py1, px2, py2 = map(int, b.xyxy[0])
                                pconf = float(b.conf[0])
                                mx1, my1, mx2, my2 = x1 + px1, y1 + py1, x1 + px2, y1 + py2
                                matricula_detectada = "matricula"
                                conf_matricula = round(pconf, 2)
                                break

                writer.writerow([
                    frame_num, label, round(conf, 2), track_id,
                    x1, y1, x2, y2,
                    matricula_detectada, conf_matricula,
                    mx1, my1, mx2, my2, texto_matricula
                ])

        if frame_num % 10 == 0:
            print(f"Procesados {frame_num}/{frame_count} fotogramas...")

vid.release()

print("\n FASE 1 COMPLETADA")
print(f" CSV guardado en: {output_csv_path}")
print(f" Conteo total: {dict(conteo)}")

# --- FASE 2: DIBUJAR EN EL VIDEO ---
print("\n Generando vídeo con anotaciones...\n")

# Leer CSV en un diccionario {frame_num: [detecciones]}
detecciones_por_frame = defaultdict(list)
with open(output_csv_path, newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        frame_num = int(row["frame"])
        detecciones_por_frame[frame_num].append(row)

vid = cv2.VideoCapture(video_path)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

frame_idx = 0
while True:
    ret, frame = vid.read()
    if not ret:
        break
    frame_idx += 1

    for row in detecciones_por_frame.get(frame_idx, []):
        x1, y1, x2, y2 = int(row["x1"]), int(row["y1"]), int(row["x2"]), int(row["y2"])
        label = row["tipo_objeto"]
        conf = float(row["confianza"])
        track_id = int(row["id_tracking"])

        color = (0, 255, 0) if label == "person" else (255, 0, 0)
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, f"{label} {conf:.2f} ID:{track_id}", (x1, y1 - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        if row["matricula_detectada"] == "matricula":
            mx1, my1, mx2, my2 = int(row["mx1"]), int(row["my1"]), int(row["mx2"]), int(row["my2"])
            pconf = float(row["conf_matricula"])
            cv2.rectangle(frame, (mx1, my1), (mx2, my2), (0, 255, 255), 2)
            cv2.putText(frame, f"plate {pconf:.2f}", (mx1, my1 - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)

    out.write(frame)
    if frame_idx % 10 == 0:
        print(f"Frame {frame_idx}/{frame_count} anotado...")

vid.release()
out.release()
cv2.destroyAllWindows()

print("\n ✅ PROCESAMIENTO COMPLETADO")
print(f" Video anotado guardado en: {output_video_path}")


In [7]:
import pandas as pd
import sys

NOMBRE_ARCHIVO_CSV =  r'resultados_video\detecciones.csv'

try:
    df = pd.read_csv(NOMBRE_ARCHIVO_CSV)

    if 'tipo_objeto' not in df.columns or 'id_tracking' not in df.columns:
        print(f"Error: El archivo '{NOMBRE_ARCHIVO_CSV}' no contiene las columnas 'tipo_objeto' o 'id_tracking'.")
        sys.exit()

    conteo_unicos = df.groupby('tipo_objeto')['id_tracking'].nunique()

    print("Conteo de IDs únicos por tipo de objeto:")
    print(conteo_unicos)

except FileNotFoundError:
    print(f"Error: No se pudo encontrar el archivo '{NOMBRE_ARCHIVO_CSV}'.")
    print("Asegúrate de que el nombre esté bien escrito y en la misma carpeta.")
except Exception as e:
    print(f"Ha ocurrido un error inesperado: {e}")

Conteo de IDs únicos por tipo de objeto:
tipo_objeto
car       248
person     48
Name: id_tracking, dtype: int64


In [8]:
import pandas as pd
import sys

NOMBRE_ARCHIVO_CSV =  r'resultados_video\detecciones_segundo_video.csv'

try:
    df = pd.read_csv(NOMBRE_ARCHIVO_CSV)

    if 'tipo_objeto' not in df.columns or 'id_tracking' not in df.columns:
        print(f"Error: El archivo '{NOMBRE_ARCHIVO_CSV}' no contiene las columnas 'tipo_objeto' o 'id_tracking'.")
        sys.exit()

    conteo_unicos = df.groupby('tipo_objeto')['id_tracking'].nunique()

    print("Conteo de IDs únicos por tipo de objeto:")
    print(conteo_unicos)

except FileNotFoundError:
    print(f"Error: No se pudo encontrar el archivo '{NOMBRE_ARCHIVO_CSV}'.")
    print("Asegúrate de que el nombre esté bien escrito y en la misma carpeta.")
except Exception as e:
    print(f"Ha ocurrido un error inesperado: {e}")

Conteo de IDs únicos por tipo de objeto:
tipo_objeto
car       233
person     10
Name: id_tracking, dtype: int64
